# Hyperopt integration prototype

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

### Config file config

In [ ]:
RAW_CONFIG_FILE_NAME = "faster_rcnn_resnet101_pets.config"
CONFIG_FILE_NAME = "tuning.config"

def change_config(params):
    with open(RAW_CONFIG_FILE_NAME, "r") as f:
        raw_config = f.read()
    config = raw_config.format(**params)
    with open(CONFIG_FILE_NAME, "w") as f:
        f.write(config)

### Start tensorflow

In [ ]:
TS_SCRIPT = "object_detection/model_main.py"
PIPELINE_CONFIG_PATH = "/home/dennis/ts-train/data/faster_rcnn_resnet101_pets.config"
MODEL_DIR = "/home/dennis/ts-train/data"
NUM_TRAIN_STEPS = 500000
NUM_EVAL_STEPS = 2000


from subprocess import run
def run_ts(params):
    try:
        run(['python3',
             TS_SCRIPT,
             '--pipeline_config_path="{path}"'.format(path=PIPELINE_CONFIG_PATH),
             '--model_dir="{path}"'.format(path=MODEL_DIR),
             '--num_train_steps={num}'.format(num=NUM_TRAIN_STEPS),
             '--num_eval_steps={num}'.format(num=NUM_EVAL_STEPS)] , check=True)
    except CalledProcessError as e:
        print("Error happened when running tensorflow:")
        print(str(e.output))

### Test model score

### Hyperopt config and main program

In [ ]:
# define search space
space4knn = {
    'initial_learning_rate': hp.uniform('initial_learning_rate', 0.000001, 0.0001)
}

# objective function
def ob_f(params):
    change_config(params)
    run_ts()
    return {'loss': -test_score(), 'status': STATUS_OK}
        

# create trial instance, which can store info during search
trials = Trials()

# run fmin to find the optimal hyperparameter
best = fmin(ob_f, space4knn, algo=tpe.suggest, max_evals=100, trials=trials)
print("best: ")
print(best)